In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf



import numpy as np
from prettytable import PrettyTable
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping,ReduceLROnPlateau
from PIL import Image, ImageChops, ImageEnhance
import os
import itertools
import seaborn as sns
import shutil
from imageio import imread
import imageio
import pandas as pd
import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from keras.applications.resnet import ResNet50, ResNet101, ResNet152
import datetime
from keras.callbacks import TensorBoard
import cv2
from skimage.transform import resize
import PIL
from numpy import save,load
from keras.optimizers import Adam,SGD
%env SM_FRAMEWORK=tf.keras


from keras import backend as K
import tensorflow_addons as tfa


from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.layers import Input, Add, Dropout,Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, concatenate,Conv2DTranspose,GlobalMaxPool2D,GlobalAveragePooling2D,UpSampling2D
import gc




PATH            = '/content/drive/Shared drives/ACloudDrive/tianchi/s2_data/data/'
TRAIN_PATH      = '/content/drive/Shared drives/ACloudDrive/tianchi/s2_data/data/train/'
TRAIN_MASK_PATH = '/content/drive/Shared drives/ACloudDrive/tianchi/s2_data/data/train_mask/'
TRAIN_ELA_PATH  = '/content/drive/Shared drives/ACloudDrive/tianchi/s2_data/data/train_ela/'
TEST_PATH       = '/content/drive/Shared drives/ACloudDrive/tianchi/s2_data/data/test/'
TEST_ELA_PATH   = '/content/drive/Shared drives/ACloudDrive/tianchi/s2_data/data/test_ela/'

CKPT_PATH       = '/content/drive/Shared drives/ACloudDrive/tianchi/s2_data/data/model/'

env: SM_FRAMEWORK=tf.keras


In [2]:
X_train = load(PATH+'npy/X_train.npy')
y_train  = load(PATH+'npy/Y_train.npy')

X_valid = load(PATH+'npy/X_valid.npy')
y_valid  = load(PATH+'npy/Y_valid.npy')

In [3]:
img_w, img_h = 512, 512
batch_size = 1
num_classes = 20

In [4]:
def AtrousSpatialPyramidPooling(model_input):
  dims = tf.keras.backend.int_shape(model_input)

  layer = tf.keras.layers.AveragePooling2D(pool_size=(dims[-3],
                                                      dims[-2]))(model_input)
  layer = tf.keras.layers.Conv2D(256, kernel_size=1, padding='same',
                                 kernel_initializer = 'he_normal')(layer)
  layer = tf.keras.layers.BatchNormalization()(layer)
  layer = tf.keras.layers.ReLU()(layer)
  out_pool = tf.keras.layers.UpSampling2D(size = (dims[-3] // layer.shape[1],
                                               dims[-2] // layer.shape[2]),
                                        interpolation = 'bilinear')(layer)
  
  layer = tf.keras.layers.Conv2D(256, kernel_size = 1,
                                   dilation_rate = 1, padding = 'same',
                                   kernel_initializer = 'he_normal',
                                   use_bias = False)(model_input)
  layer = tf.keras.layers.BatchNormalization()(layer)
  out_1 = tf.keras.layers.ReLU()(layer)

  layer = tf.keras.layers.Conv2D(256, kernel_size = 3,
                                   dilation_rate = 6, padding = 'same', 
                                   kernel_initializer = 'he_normal',
                                   use_bias = False)(model_input)
  layer = tf.keras.layers.BatchNormalization()(layer)
  out_6 = tf.keras.layers.ReLU()(layer)

  layer = tf.keras.layers.Conv2D(256, kernel_size = 3,
                                   dilation_rate = 12, padding = 'same',
                                   kernel_initializer = 'he_normal',
                                   use_bias = False)(model_input)
  layer = tf.keras.layers.BatchNormalization()(layer)
  out_12 = tf.keras.layers.ReLU()(layer)

  layer = tf.keras.layers.Conv2D(256, kernel_size = 3,
                                   dilation_rate = 18, padding = 'same',
                                   kernel_initializer = 'he_normal',
                                   use_bias = False)(model_input)
  layer = tf.keras.layers.BatchNormalization()(layer)
  out_18 = tf.keras.layers.ReLU()(layer)

  layer = tf.keras.layers.Concatenate(axis = -1)([out_pool, out_1,
                                                    out_6, out_12,
                                                    out_18])

  layer = tf.keras.layers.Conv2D(256, kernel_size = 1,
                                   dilation_rate = 1, padding = 'same',
                                   kernel_initializer = 'he_normal',
                                   use_bias = False)(layer)
  layer = tf.keras.layers.BatchNormalization()(layer)
  model_output = tf.keras.layers.ReLU()(layer)
  return model_output

In [5]:
def DeeplabV3Plus(nclasses = 1):
  model_input = tf.keras.Input(shape=(img_h,img_w,3))
  resnet50 = tf.keras.applications.ResNet50(weights = None,
                                            include_top = False,
                                            input_tensor = model_input)
  layer = resnet50.get_layer('conv4_block6_2_relu').output
  layer = AtrousSpatialPyramidPooling(layer)
  input_a = tf.keras.layers.UpSampling2D(size = (img_h // 4 // layer.shape[1],
                                                 img_w // 4 // layer.shape[2]),
                                          interpolation = 'bilinear')(layer)

  input_b = resnet50.get_layer('conv2_block3_2_relu').output
  input_b = tf.keras.layers.Conv2D(48, kernel_size = (1,1), padding = 'same',
                                   kernel_initializer = tf.keras.initializers.he_normal(),
                                   use_bias = False)(input_b)
  input_b = tf.keras.layers.BatchNormalization()(input_b)
  input_b = tf.keras.layers.ReLU()(input_b)

  layer = tf.keras.layers.Concatenate(axis = -1)([input_a, input_b])

  layer = tf.keras.layers.Conv2D(256, kernel_size = 3,
                                   padding = 'same', activation = 'relu',
                                   kernel_initializer = tf.keras.initializers.he_normal(),
                                   use_bias = False)(layer)
  layer = tf.keras.layers.BatchNormalization()(layer)
  layer = tf.keras.layers.ReLU()(layer)
  layer = tf.keras.layers.Conv2D(256, kernel_size =3,
                                   padding = 'same', activation = 'relu',
                                   kernel_initializer = tf.keras.initializers.he_normal(),
                                   use_bias = False)(layer)
  layer = tf.keras.layers.BatchNormalization()(layer)
  layer = tf.keras.layers.ReLU()(layer)
  layer = tf.keras.layers.UpSampling2D(size = (img_h // layer.shape[1],
                                                 img_w // layer.shape[2]),
                                          interpolation = 'bilinear')(layer)
  model_output = tf.keras.layers.Conv2D(num_classes, kernel_size = (1,1),
                                   padding = 'same')(layer)
  return tf.keras.Model(inputs = model_input, outputs = model_output)

model = DeeplabV3Plus()
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
# Dice_Coeff or F1 score
def metric(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1)
bs=4
epochs=20
step_per_epoch=steps_per_epoch=(len(X_train) // bs)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=loss,metrics=metric)

from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
import time
                  
t= time.strftime("%Y%m%d-%H%M%S", time.localtime())
### MODELCHECKPOINT CALLBACK
save = ModelCheckpoint(CKPT_PATH+'{}_model.h5'.format(t), verbose=1, save_best_only=True, save_weights_only=True)


### REDUCES LR WHEN METRTIC IS NOT IMPROVING
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1, min_delta =1e-4, min_lr=0.00001, mode='min')

### LEARNING RATE SCHEDULER
def scheduler(epoch):

  if epoch < 10:
     return 0.001
  else:
     return float(0.001 * tf.math.exp(0.1 * (10 - epoch)))
lr_schedule = LearningRateScheduler(scheduler) 

### EARLY STOPPING
early_stopping = EarlyStopping(patience=10, verbose=1)

# Data Generator and augmentation
data_gen_args = dict(horizontal_flip=True,vertical_flip=True)
image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

seed = 42
bs = 1
nb_epochs = 20

image_generator = image_datagen.flow(X_train, seed=seed, batch_size=bs, shuffle=True)
mask_generator = mask_datagen.flow(y_train, seed=seed, batch_size=bs, shuffle=True)

# Just zip the two generators to get a generator that provides augmented images and masks at the same time
train_generator = zip(image_generator, mask_generator)

results = model.fit_generator(train_generator, steps_per_epoch=(len(X_train) // bs), epochs=nb_epochs,validation_data=(X_valid, y_valid),callbacks=[save,lr_schedule,reduce_lr])